In [15]:
import time
import threading
from datetime import datetime
from typing import Dict, Optional
import pandas as pd
import warnings

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.wrapper import *
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import *
from enum import Enum
from datetime import datetime, timezone

In [16]:
def donchian_channel(df: pd.DataFrame, period: int = 30) -> pd.DataFrame:

    df["upper"] = df["high"].rolling(window=period).max()

    df["lower"] = df["low"].rolling(window=period).min()

    df["mid"] = (df["upper"] + df["lower"]) / 2

    return df
class DiffLevelEnum(Enum):
    NONE = "NONE"
    LOW = "LOW"
    MEDIUM = "MEDIUM"
    HIGH = "HIGH"
    SUPER_HIGH = "SUPER_HIGH"
class TradingApp(EClient, EWrapper):

    def __init__(self) -> None:

        EClient.__init__(self, self)
        self.df_saved_ticks  = pd.DataFrame(columns=[
            'Time', 'TickAttriBidAsk', 'AskPastHigh', 'PriceBid',
            'PriceAsk', 'SizeBid', 'SizeAsk', 'TimeFormatted'
        ])
        self.data: Dict[int, pd.DataFrame] = {}
        self.nextOrderId: Optional[int] = None

    def error(self, reqId: int, errorCode: int, errorString: str) -> None:

        print(f"Error: {reqId}, {errorCode}, {errorString}")

    def nextValidId(self, orderId: int) -> None:

        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:

        self.data[reqId] = pd.DataFrame(columns=["time", "high", "low", "close"])
        self.data[reqId].set_index("time", inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(5)
        return self.data[reqId]

    def historicalTicks(self, reqId: int, ticks: ListOfHistoricalTickLast, done: bool):
        print("no esta viniendo nada para mi ticks: " + str(len(ticks)))
    def historicalTicks(self, reqId: int, ticks: ListOfHistoricalTickBidAsk, done: bool):
        print("no esta viniendo nada para mi tick last : " + str(len(ticks)))

        # for tick in ticks:
        #     print("HistoricalTickLast. ReqId:", reqId, tick)

    def historicalData(self, reqId: int, bar: BarData) -> None:

        df = self.data[reqId]

        df.loc[
            pd.to_datetime(bar.date, unit="s"),
            ["high", "low", "close"]
        ] = [bar.high, bar.low, bar.close]

        df = df.astype(float)

        self.data[reqId] = df

    @staticmethod
    def get_contract(symbol: str) -> Contract:

        contract = Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract
    @staticmethod
    def get_forex_contract(pair: str) -> Contract:
        """
        Devuelve un contrato de tipo CASH para operar en Forex.

        Ejemplo: pair='EURUSD'
        """
        contract = Contract()
        contract.symbol = 'EUR'  # 'EUR'
        contract.secType = "CASH"
        contract.exchange = "IDEALPRO"
        contract.currency = 'USD'  # 'USD'
        return contract

    def place_order(self, contract: Contract, action: str, order_type: str, quantity: int) -> None:

        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity

        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1
        print("Order placed")
    def historicalTicksBidAsk(self, reqId: int, ticks: ListOfHistoricalTickBidAsk, done: bool):
        print("ticks bid ask: " + str(len(ticks)) +" : "+ str(done) + " : " +str(reqId))

        parsed_list = []
        for tick in ticks:
            raw = str(tick)

            # Separar y convertir en dict
            parsed = {}
            for pair in raw.split(", "):
                if ": " in pair:
                    key, value = pair.split(": ", 1)
                    try:
                        parsed[key] = float(value) if '.' in value else int(value)
                    except ValueError:
                        parsed[key] = value  # por si hay strings
            parsed_list.append(parsed)
        df = pd.DataFrame(parsed_list)
        #

        self.data[10899] = df
# 20250528-04:15:00
    def get_historical_data_by_tick(self, contract: Contract, start_time : str, end_time : str) -> pd.DataFrame:#"20250528-04:15:00"
        self.reqHistoricalTicks(10899, contract, start_time, end_time, 1000, "BID_ASK", 1, False, [])
        self.data[10899] = pd.DataFrame(columns=['Time', 'TickAttriBidAsk', 'AskPastHigh', 'PriceBid', 'PriceAsk', 'SizeBid', 'SizeAsk'])
        self.data[10899].set_index("Time", inplace=True)
        time.sleep(1)

        return self.data[10899]

    def get_ticks_per_bar(self, start_time : str, end_time : str):
        contract_eurusd = self.get_forex_contract("EURUSD")
        stop_time = end_time
        stop_time_dt = pd.to_datetime(stop_time, utc=True)
        start_time_dt = pd.to_datetime(start_time, utc=True)
        counter = 0
        boolean =True
        while boolean:
            df = self.get_historical_data_by_tick(contract_eurusd, start_time, end_time)
            counter += 1
            print("largoo "+str(len(df)))
            if len(df) > 1:
                boolean =False
            elif counter > 5:
                print(counter)
                return self.df_saved_ticks
        print(counter)
        df['TimeFormatted'] = pd.to_datetime(df["'Time'"], unit='s', utc=True)

        #te clavas 2 segundos

        max_time = df["TimeFormatted"].max()
        min_time = df["TimeFormatted"].min()
        print(f"max {max_time} min {min_time}")
        pd_end_time=pd.to_datetime(end_time, utc=True)
        while max_time < pd_end_time:
            new_start_time=max_time.strftime('%Y%m%d-%H:%M:%S') #Aca seguro hay que matarle algunos repetidos
            df_aux = self.get_historical_data_by_tick(contract_eurusd, new_start_time, end_time)
            df_aux['TimeFormatted'] = pd.to_datetime(df_aux["'Time'"], unit='s', utc=True)
            max_time = df_aux["TimeFormatted"].max()
            df = pd.concat([df,df_aux], ignore_index=True)

        df_filtered_1min = df[df['TimeFormatted'] <= stop_time_dt]
        df_filtered_1min = df_filtered_1min[df_filtered_1min['TimeFormatted'] >= start_time_dt]
        print(str(len(df)) + " : " + str(len(df_filtered_1min)))
        print(str(stop_time_dt) + " -- " + stop_time)

        return df_filtered_1min
    def convert_values_to_str(self, sum_ask,sum_bid):
        difference = sum_bid - sum_ask

        if(difference == 0):
            diff_level = "NONE"
        elif (abs(difference) < 30 * 1000000):
            diff_level = "LOW"
        elif (abs(difference) < 60 * 1000000):
            diff_level = "MEDIUM"
        elif (abs(difference) < 1500 * 1000000):
            diff_level = "HIGH"
        else:
            diff_level = "SUPER_HIGH"
        return diff_level
    @staticmethod
    def format_int_to_string(num):
        if num >= 1_000_000_000:
            return f"{num / 1_000_000_000:.3f} B"
        elif num >= 1_000_000:
            return f"{num / 1_000_000:.3f} M"
        elif num >= 1_000:
            return f"{num / 1_000:.3f} K"
        else:
            return str(num)

In [17]:
app = TradingApp()

app.connect("127.0.0.1", 7497, clientId=5)

threading.Thread(target=app.run, daemon=True).start()

while True:
    if isinstance(app.nextOrderId, int):
        print("connected")
        break
    else:
        print("waiting for connection")
        time.sleep(1)


waiting for connection
Error: -1, 2104, La conexión al centro de datos funciona correctamente:usfarm.nj
Error: -1, 2104, La conexión al centro de datos funciona correctamente:usfuture
Error: -1, 2104, La conexión al centro de datos funciona correctamente:usopt.nj
Error: -1, 2104, La conexión al centro de datos funciona correctamente:cashfarm
Error: -1, 2104, La conexión al centro de datos funciona correctamente:usfarm
Error: -1, 2106, La conexión al centro de datos HMDS funciona correctamente:cashhmds
Error: -1, 2106, La conexión al centro de datos HMDS funciona correctamente:ushmds
Error: -1, 2158, La conexión de la granja de datos "sec-def" funciona correctamente:secdefnj
connected
ticks bid ask: 1024 : True : 10899
ticks bid ask: 1009 : True : 10899
ticks bid ask: 1034 : True : 10899


In [22]:

date_from, date_to = "20250612-20:13:00", "20250612-20:13:59"
df_filtered_1min = app.get_ticks_per_bar(date_from, date_to)
if len(df_filtered_1min) != 0:
    sum_ask = df_filtered_1min["'SizeAsk'"].sum()
    sum_bid = df_filtered_1min["'SizeBid'"].sum()
    difference = sum_bid - sum_ask
    date_from_db_format = pd.to_datetime(date_from, utc=True)
    date_to_db_format = pd.to_datetime(date_to  , utc=True)
    diff_str = app.convert_values_to_str(sum_ask,sum_bid)
    count_tick = len(df_filtered_1min)
    updated_at = datetime.now(timezone.utc)
    nw_day = datetime.fromisoformat(str(date_from_db_format)).weekday()  >= 5
    db_line = f"""
    symbolid: 1 |datefrom {date_from_db_format} | dateto {date_to_db_format} |
    sumask {str(sum_ask)} | sumbid {str(sum_bid)} | difference {str(difference)} |
    sumask_str {TradingApp.format_int_to_string(sum_ask)} | sumask_str {TradingApp.format_int_to_string(sum_bid)} | difference {TradingApp.format_int_to_string(difference)} |
    count_tick {count_tick} | SUCCES | updated_at {updated_at} | diff level enum {diff_str} | retry_count | dow | NW_DAY {nw_day}
    """
    print(db_line)

largoo 1004
1
max 2025-06-12 20:21:17+00:00 min 2025-06-12 20:12:59+00:00
1004 : 128
2025-06-12 20:13:59+00:00 -- 20250612-20:13:59

    symbolid: 1 |datefrom 2025-06-12 20:13:00+00:00 | dateto 2025-06-12 20:13:59+00:00 |
    sumask 440600000 | sumbid 277590000 | difference -163010000 |
    sumask_str 440.600 M | sumask_str 277.590 M | difference -163010000 |
    count_tick 128 | SUCCES | updated_at 2025-06-13 17:33:10.098042+00:00 | diff level enum HIGH | retry_count | dow | NW_DAY False
    


In [235]:
]print(sum_ask)
df_filtered_1min

6108900000


,'Time','TickAttriBidAsk',AskPastHigh,'PriceBid','PriceAsk','SizeBid','SizeAsk',TimeFormatted
1,1748403300,BidPastLow: 0,0,1.13025,1.13028,2000000,6000000,2025-05-28 03:35:00+00:00
2,1748403300,BidPastLow: 0,0,1.13025,1.13028,2000000,5000000,2025-05-28 03:35:00+00:00
3,1748403300,BidPastLow: 0,0,1.13026,1.13028,1000000,4000000,2025-05-28 03:35:00+00:00
4,1748403300,BidPastLow: 0,0,1.13026,1.13028,2000000,3000000,2025-05-28 03:35:00+00:00
5,1748403300,BidPastLow: 0,0,1.13026,1.13028,2000000,2000000,2025-05-28 03:35:00+00:00
...,...,...,...,...,...,...,...,...
2696,1748403359,BidPastLow: 0,0,1.13025,1.13030,1000000,5000000,2025-05-28 03:35:59+00:00
2697,1748403359,BidPastLow: 0,0,1.13024,1.13030,4000000,5000000,2025-05-28 03:35:59+00:00
2698,1748403359,BidPastLow: 0,0,1.13024,1.13030,4000000,4000000,2025-05-28 03:35:59+00:00
2699,1748403359,BidPastLow: 0,0,1.13024,1.13030,3000000,3000000,2025-05-28 03:35:59+00:00


In [14]:

app.disconnect()

In [ ]:
import requests

max_to_process = 720
url = f"http://localhost:8080/ib/v3/processData/{max_to_process}"

headers = {
    "Content-Type": "application/json; charset=utf-8",
    "Accept": "application/json"
}

response = requests.post(url, headers=headers)

if response.ok:
    print("Respuesta:", response.json())
else:
    print("Error:", response.status_code, response.text)